In [32]:
import geopandas as gpd
import pandas as pd

In [71]:
mumbai_wards=gpd.read_file("../data/raw/ward_boundaries/mumbai_wards.geojson")
mumbai_wards=mumbai_wards.to_crs("EPSG:4326")
mumbai_wards.head()

,gid,objectid_1,name,shape_leng,shape_area,geometry
0,1,1,A,23100.418607,1.120911e+07,"POLYGON ((72.84681 18.94803, 72.84673 18.9476,..."
1,2,2,B,8322.589466,2.658217e+06,"POLYGON ((72.84938 18.9634, 72.84913 18.96275,..."
2,3,3,C,6389.304749,1.913026e+06,"POLYGON ((72.83366 18.94747, 72.83361 18.94743..."
3,4,4,D,16048.789624,8.223427e+06,"POLYGON ((72.82625 18.96169, 72.82625 18.96168..."
4,5,5,E,15964.521869,7.273882e+06,"POLYGON ((72.85003 18.98002, 72.85012 18.97998..."


In [83]:
ward_centroids=mumbai_wards.copy()
ward_centroids["geometry"] = ward_centroids.geometry.centroid
ward_centroids=ward_centroids[["name","geometry"]]
ward_centroids.head()

C:\Users\Aryan\AppData\Local\Temp\ipykernel_7744\3818709412.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ward_centroids["geometry"] = ward_centroids.geometry.centroid


,name,geometry
0,A,POINT (72.825 18.92314)
1,B,POINT (72.84014 18.95636)
2,C,POINT (72.82737 18.95111)
3,D,POINT (72.81005 18.96234)
4,E,POINT (72.8365 18.97398)


In [90]:
rain24=pd.read_csv("../data/processed/rainfall_2024.csv")

rain24.columns

Index(['time', 'lat', 'lon', 'rf'], dtype='object')

In [91]:
rain24_num=rain24[(rain24["lat"] >=18.6) & (rain24["lat"] <=19.4) & (rain24["lon"]>=72.7)&(rain24["lon"] <=73.1)]

In [92]:
rain24_gdf=gpd.GeoDataFrame(rain24,geometry=gpd.points_from_xy(rain24.lon,rain24.lat),crs="EPSG:4326")

In [93]:
rain24_ward=gpd.sjoin_nearest(ward_centroids,rain24_gdf,how="left",distance_col="dist_to_grid")

c:\Users\Aryan\AppData\Local\Programs\Python\Python313\Lib\site-packages\geopandas\array.py:407: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [100]:
rain24_ward[["name","rf","dist_to_grid"]].head()

,name,rf,dist_to_grid
0,A,0.0,0.107386
0,A,0.0,0.107386
0,A,0.0,0.107386
0,A,0.0,0.107386
0,A,0.0,0.107386


In [101]:
ward_rain_24 = rain24_ward.groupby("name").agg(
    avg_rainfall=("rf", "mean"),
    max_rainfall=("rf", "max")
).reset_index()

ward_rain_24["year"] = 2024

In [102]:

mumbai_wards.total_bounds

array([72.77633295, 18.89395643, 72.9797315 , 19.27017667])

In [103]:
rain24_gdf.total_bounds

array([ 66.5,   6.5, 100. ,  38.5])

In [104]:
rain24_ward["name"].value_counts()

name
A      366
B      366
C      366
D      366
E      366
F/S    366
G/S    366
F/N    366
G/N    366
N      366
R/C    366
S      366
T      366
K/W    366
R/N    366
M/E    366
M/W    366
H/E    366
K/E    366
P/S    366
P/N    366
R/S    366
H/W    366
L      366
Name: count, dtype: int64

In [106]:
rain25=pd.read_csv("../data/processed/rainfall_2025.csv")

rain25.columns

Index(['TIME', 'LATITUDE', 'LONGITUDE', 'RAINFALL'], dtype='object')

In [108]:
rain25_gdf=gpd.GeoDataFrame(rain25,geometry=gpd.points_from_xy(rain25.LONGITUDE,rain25.LATITUDE),crs="EPSG:4326")

In [111]:
rain25_ward=gpd.sjoin_nearest(ward_centroids,rain25_gdf,how="left",distance_col="dist_to_grid")

c:\Users\Aryan\AppData\Local\Programs\Python\Python313\Lib\site-packages\geopandas\array.py:407: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [113]:
rain25_ward[["name","RAINFALL","dist_to_grid"]].head()

,name,RAINFALL,dist_to_grid
0,A,1.190568,0.107386
0,A,0.000000,0.107386
0,A,100.071200,0.107386
0,A,11.335196,0.107386
0,A,5.370557,0.107386


In [114]:

ward_rain_25 = rain25_ward.groupby("name").agg(
    avg_rainfall=("RAINFALL", "mean"),
    max_rainfall=("RAINFALL", "max")
).reset_index()

ward_rain_25["year"] = 2025

In [109]:
rain25_gdf.total_bounds

array([ 66.5,   6.5, 100. ,  38.5])

In [110]:
rain24_ward["name"].value_counts()

name
A      366
B      366
C      366
D      366
E      366
F/S    366
G/S    366
F/N    366
G/N    366
N      366
R/C    366
S      366
T      366
K/W    366
R/N    366
M/E    366
M/W    366
H/E    366
K/E    366
P/S    366
P/N    366
R/S    366
H/W    366
L      366
Name: count, dtype: int64

In [115]:
rain_all=pd.concat([ward_rain_24,ward_rain_25])

In [116]:
ward_rain = rain_all.groupby("name").agg(
    avg_rainfall=("avg_rainfall", "mean"),
    max_rainfall=("max_rainfall", "max")
).reset_index()

ward_rain.rename(columns={"name": "ward"}, inplace=True)

In [117]:
ward_rain.to_csv("../data/processed/ward_rainfall_features.csv",index=False)